# We will demonstrate skpref on discrete choice data

We wil use the swissmetro dataset available to download on https://transp-or.epfl.ch/pythonbiogeme/examples_swissmetro.html
This dataset tracks 470 respondents on which transportation alternative they have taken. There are 3 options in general: train, car and swissmetro.
More details on the original use of the dataset can be found here: http://strc.ch/2001/bierlaire1.pdf

In [1]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import sys
sys.path.insert(0, "../..")
from skpref.base import ClassificationReducer
from skpref.random_utility import BradleyTerry
from skpref.task import ChoiceTask
from skpref.metrics import f1_score, log_loss, log_loss_compare_with_t_test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from copy import deepcopy
from sklearn.linear_model import LogisticRegression

In [2]:
swissmetro = pd.read_csv("data/swissmetro.dat", sep='\t')
swissmetro.head()

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,MALE,INCOME,GA,ORIGIN,DEST,TRAIN_AV,CAR_AV,SM_AV,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,0,2,0,2,1,1,1,1,130,36,60,63,42,20,0,90,84,2


Looking at the data we can see that each row represents a choice.
The full explanations of variables can be found here: https://transp-or.epfl.ch/pythonbiogeme/examples/swissmetro/swissmetro.pdf

# We will need to make a couple of changes to the format of this table for skpref to read it easier. <br>

1) In this table the availability of alternatives is marked by TRAIN_AV, CAR_AV, SM_AV which indicate with 1 if the alternative is available and 0 otherwise. We need to convert these to a column that contains a list of alternatives for each row.

2) The choices are indicated by the CHOICE column, which contains 0 for unknown (we will dropping these), 1 for Train, 2 for Swissmetro and 3 for a Car usage. We need to name these explicitly. We could just create a list of alternatives called 1,2,3 in step 1 which would bypass step 2, however, we prefer the clarity of having the alternatives named explicitly.

# We will use a subset of the available features to make the skpref demo easier
Under normal circumstances we would use one-hot encoding on a lot of the binary features before training a serious model, however, to make the demo simple, we will only use the travel time and cost features. Users can of course do whatever feature transformations they like before fitting a model.
To build a classifier based on travel time and costs, we first need to split some columns from the swissmetro dataset into a secondary table.

In [3]:
train_vals = swissmetro[['TRAIN_TT', 'TRAIN_CO']].copy()
train_vals.columns = ['Travel Time', 'Cost']
train_vals.reset_index(inplace=True)
train_vals['alternative'] = 'Train'
swissmetro_vals = swissmetro[['SM_TT', 'SM_CO']].copy()
swissmetro_vals.columns = ['Travel Time', 'Cost']
swissmetro_vals.reset_index(inplace=True)
swissmetro_vals['alternative'] = 'Swiss Metro'
car_vals = swissmetro[['CAR_TT', 'CAR_CO']].copy()
car_vals.columns = ['Travel Time', 'Cost']
car_vals.reset_index(inplace=True)
car_vals['alternative'] = 'Car'
dummy_secondary_table = (
    train_vals.append(swissmetro_vals.append(car_vals))
).sort_values('index')
dummy_secondary_table.rename(columns={'index': 'merge_index'}, inplace=True)
dummy_secondary_table.head()

,merge_index,Travel Time,Cost,alternative
0,0,112,48,Train
0,0,63,52,Swiss Metro
0,0,117,65,Car
1,1,103,48,Train
1,1,60,49,Swiss Metro


In [4]:
binary_concats = (swissmetro.TRAIN_AV.astype(str) + 
                  swissmetro.CAR_AV.astype(str)  + 
                  swissmetro.SM_AV.astype(str)
                 )

alts = []
for i in binary_concats.values:
    if i == '111':
        alts.append(['Train', 'Car', 'Swiss Metro'])
    elif i == '100':
        alts.append(['Train'])
    elif i == '000':
        alts.append(['None'])
    elif i == '010':
        alts.append(['Car'])
    elif i == '001':
        alts.append(['Swiss Metro'])
    elif i == '101':
        alts.append(['Train', 'Swiss Metro'])
    elif i == '110':
        alts.append(['Train', 'Car'])
    elif i == '011':
        alts.append(['Car', 'Swiss Metro'])
        
swissmetro['alternatives'] = alts
        
swissmetro['chosen'] = np.where(swissmetro.CHOICE.values==1, 'Train',
                       np.where(swissmetro.CHOICE.values==2, 'Swiss Metro',
                       np.where(swissmetro.CHOICE.values==3, 'Car',
                                'unknown')))
swissmetro = swissmetro[swissmetro.CHOICE != 0].copy()
swissmetro = swissmetro.reset_index()[['alternatives', 'chosen', 'index']]
swissmetro.rename(columns={'index': 'merge_index'}, inplace=True)
swissmetro.head()

,alternatives,chosen,merge_index
0,"[Train, Car, Swiss Metro]",Swiss Metro,0
1,"[Train, Car, Swiss Metro]",Swiss Metro,1
2,"[Train, Car, Swiss Metro]",Swiss Metro,2
3,"[Train, Car, Swiss Metro]",Swiss Metro,3
4,"[Train, Car, Swiss Metro]",Swiss Metro,4


In [5]:
swissmetro.alternatives.values

array([list(['Train', 'Car', 'Swiss Metro']),
       list(['Train', 'Car', 'Swiss Metro']),
       list(['Train', 'Car', 'Swiss Metro']), ...,
       list(['Train', 'Car', 'Swiss Metro']),
       list(['Train', 'Car', 'Swiss Metro']),
       list(['Train', 'Car', 'Swiss Metro'])], dtype=object)

In [6]:
train, test = train_test_split(swissmetro, random_state=1, test_size=0.1)

swiss_metro_train = ChoiceTask(train, 'alternatives', 'chosen',
                               features_to_use=['Travel Time', 'Cost'],
                               secondary_table=dummy_secondary_table,
                               secondary_to_primary_link={
                                   'merge_index': 'merge_index',
                                   'alternative': 'alternatives'
                               }
                              )

swiss_metro_test = ChoiceTask(test, 'alternatives', 'chosen',
                              features_to_use=['Travel Time', 'Cost'],
                              secondary_table=dummy_secondary_table,
                              secondary_to_primary_link={
                                   'merge_index': 'merge_index',
                                   'alternative': 'alternatives'
                               }
                             )

In [7]:
my_log_red = ClassificationReducer(LogisticRegression(solver='lbfgs'))
my_log_red.fit_task(swiss_metro_train)
preds = my_log_red.predict_proba_task(swiss_metro_test,
                                      ['Swiss Metro','Train', 'Car'])

In [8]:
preds

{'Swiss Metro': array([0.4873467 , 0.55661961, 0.39863582, ..., 0.46693185, 0.3517729 ,
        0.51405707]),
 'Train': array([0.30457664, 0.44643913, 0.16638042, ..., 0.35997311, 0.29034294,
        0.2626234 ]),
 'Car': array([0.33291693, 0.        , 0.19868472, ..., 0.45209947, 0.36246787,
        0.25068403])}

In [9]:
outocme_preds = my_log_red.predict_task(swiss_metro_test)
print(outocme_preds.top_input_data[:5])
print(outocme_preds.boot_input_data[:5])

['Swiss Metro' 'Swiss Metro' 'Swiss Metro' 'Swiss Metro' 'Swiss Metro']
[array(['Car', 'Train'], dtype='<U11') array(['Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')]


In [10]:
# Fit a Bradley Terry model with no features
swiss_metro_train_BT = ChoiceTask(train.drop('merge_index', axis=1), 
                                  'alternatives',
                                  'chosen', features_to_use=None)

swiss_metro_test_BT = ChoiceTask(test.drop('merge_index', axis=1), 
                                 'alternatives',
                                 'chosen', features_to_use=None)

In [11]:
my_BT_red = BradleyTerry(method='BFGS', alpha=1e-5)
my_BT_red.fit_task(swiss_metro_train_BT)
preds = my_BT_red.predict_proba_task(swiss_metro_test_BT,
                                     ['Swiss Metro','Train', 'Car'])

In [12]:
preds

{'Swiss Metro': array([0.54530381, 0.8344241 , 0.54530381, ..., 0.54530381, 0.54530381,
        0.54530381]),
 'Train': array([0.10820537, 0.1655759 , 0.10820537, ..., 0.10820537, 0.10820537,
        0.10820537]),
 'Car': array([0.34649083, 0.        , 0.34649083, ..., 0.34649083, 0.34649083,
        0.34649083])}

In [13]:
choice_preds = my_BT_red.predict_task(swiss_metro_test_BT)
print(choice_preds.top_input_data[:5])
print(choice_preds.boot_input_data[:5])

['Swiss Metro' 'Swiss Metro' 'Swiss Metro' 'Swiss Metro' 'Swiss Metro']
[array(['Car', 'Train'], dtype='<U11') array(['Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')
 array(['Car', 'Train'], dtype='<U11')]


In [14]:
# Fit a Bradley Terry model with features
# Reducing training set because on my PC this gives a memory error
swiss_metro_train_BT_feats = ChoiceTask(
    train.sample(frac=0.5), 'alternatives', 'chosen',
    secondary_table=dummy_secondary_table,
    secondary_to_primary_link={
        'merge_index': 'merge_index',
        'alternative': 'alternatives'
    },
    features_to_use=['Travel Time', 'Cost'])

swiss_metro_test = ChoiceTask(test, 'alternatives', 'chosen',
                              features_to_use=['Travel Time', 'Cost'],
                              secondary_table=dummy_secondary_table,
                              secondary_to_primary_link={
                                   'merge_index': 'merge_index',
                                   'alternative': 'alternatives'
                               }
                             )

my_BT_red_feats = BradleyTerry(method='BFGS', alpha=1e-5)
my_BT_red_feats.fit_task(swiss_metro_train_BT_feats)

In [15]:
swiss_metro_test = ChoiceTask(test, 'alternatives', 'chosen',
                              features_to_use=['Travel Time', 'Cost'],
                              secondary_table=dummy_secondary_table,
                              secondary_to_primary_link={
                                   'merge_index': 'merge_index',
                                   'alternative': 'alternatives'
                               }
                             )
preds = my_BT_red_feats.predict_proba_task(swiss_metro_test,
                                      ['Swiss Metro','Train', 'Car'])

In [16]:
my_BT_red_feats.predict_task(swiss_metro_test).top_input_data

array(['Swiss Metro', 'Swiss Metro', 'Swiss Metro', ..., 'Car', 'Car',
       'Swiss Metro'], dtype=object)

In [17]:
preds

{'Swiss Metro': array([0.56275278, 0.77173434, 0.67096487, ..., 0.40555301, 0.37622761,
        0.6920563 ]),
 'Train': array([0.1116056 , 0.22826566, 0.07890942, ..., 0.11959485, 0.13415101,
        0.09350961]),
 'Car': array([0.32564162, 0.        , 0.25012571, ..., 0.47485214, 0.48962138,
        0.21443408])}

In [18]:
ind_trans_preds = my_BT_red_feats.predict_proba_task(swiss_metro_test,
                                   ['Swiss Metro','Train', 'Car'],
                                   aggregation_method='independent transitive')
ind_trans_preds

{'Swiss Metro': array([0.62643345, 0.77173434, 0.73777348, ..., 0.4273645 , 0.39028417,
        0.77002546]),
 'Train': array([0.05005995, 0.22826566, 0.02856546, ..., 0.05504243, 0.06887847,
        0.04138344]),
 'Car': array([0.3235066 , 0.        , 0.23366106, ..., 0.51759307, 0.54083736,
        0.18859109])}

In [19]:
preds_outcome_ind_trans = my_BT_red_feats.predict_task(swiss_metro_test,
                             aggregation_method='independent transitive')
preds_outcome_Luce = my_BT_red_feats.predict_task(swiss_metro_test)

In [20]:
dummy_secondary_table.groupby('alternative').mean()

,merge_index,Travel Time,Cost
alternative,,,
Car,5363.5,123.795209,78.742077
Swiss Metro,5363.5,87.466350,670.340697
Train,5363.5,166.626025,514.335477


In [21]:
my_BT_red_feats.bt_with_feats.print_summaries()



Number of Parameters                                                      5
Number of Observations                                                 8883
Null Log-Likelihood                                            -6157.226405
Fitted Log-Likelihood                                          -4634.988551
Rho-Squared                                                        0.247228
Rho-Bar-Squared                                                    0.246416
Estimation Message        Desired error not necessarily achieved due to ...
dtype: object
             parameters     std_err    t_stats       p_values  robust_std_err  \
Cost           0.000274    0.000031   8.724925   2.663564e-18        0.000034   
Travel Time   -0.011972    0.000550 -21.787243  3.065593e-105        0.000938   
Car            0.380587  129.099445   0.002948   9.976478e-01             NaN   
Swiss Metro    0.154561  129.099446   0.001197   9.990448e-01             NaN   
Train         -0.535147  129.099446  -0.004145 

In [26]:
print(f"The F1 score of the Luce aggregation was \
{f1_score(swiss_metro_test.subset_vec, preds_outcome_Luce): .3}")
print(f"The F1 score of the generic aggregation was \
{f1_score(swiss_metro_test.subset_vec, preds_outcome_ind_trans):.3}")

The F1 score of the Luce aggregation was  0.595
The F1 score of the generic aggregation was 0.595


In [27]:
random_probs = {
    'Swiss Metro': np.ones(len(test)) * (1/3),
    'Train': np.ones(len(test)) * (1/3),
    'Car': np.ones(len(test)) * (1/3)
}
print(f"The log loss for each alternative in the Luce aggregation was \
{log_loss(swiss_metro_test.subset_vec, preds)}")
print(f"The log loss for each alternative in the generic aggregation was \
{log_loss(swiss_metro_test.subset_vec, ind_trans_preds)}")
print(f"The log loss for each alternative assigning random probability was \
{log_loss(swiss_metro_test.subset_vec, random_probs)}")

The log loss for each alternative in the Luce aggregation was {'Swiss Metro_log_loss': 0.7023664871593164, 'Train_log_loss': 0.383649993519745, 'Car_log_loss': 0.5174650954050872}
The log loss for each alternative in the generic aggregation was {'Swiss Metro_log_loss': 0.711234682056963, 'Train_log_loss': 0.39622633781562233, 'Car_log_loss': 0.5207606018292977}
The log loss for each alternative assigning random probability was {'Swiss Metro_log_loss': 0.7973001747306709, 'Train_log_loss': 0.5037471710233804, 'Car_log_loss': 0.6084951591303872}


In [31]:
print(f"The t-test for H0: Luce aggregation = Generic aggregation \
{log_loss_compare_with_t_test(swiss_metro_test.subset_vec, preds, ind_trans_preds)}")
print(f"The t-test for H0: Generic aggregation = random probability \
{log_loss_compare_with_t_test(swiss_metro_test.subset_vec, ind_trans_preds, random_probs)}")

The t-test for H0: Luce aggregation = Generic aggregation {'Swiss Metro': Ttest_relResult(statistic=2.3844799687642295, pvalue=0.01727655183381647), 'Train': Ttest_relResult(statistic=1.8876865507212632, pvalue=0.059338206864608084), 'Car': Ttest_relResult(statistic=1.4551854443597005, pvalue=0.14591095911650695)}
The t-test for H0: Generic aggregation = random probability {'Swiss Metro': Ttest_relResult(statistic=3.505669485303979, pvalue=0.00047431133111994943), 'Train': Ttest_relResult(statistic=5.593686571881314, pvalue=2.821407818283642e-08), 'Car': Ttest_relResult(statistic=5.825562533722975, pvalue=7.52159149793693e-09)}


../..\skpref\metrics\_classification.py:190: RuntimeWarning: divide by zero encountered in log
  logged = np.log(predicted1[_alternative])
../..\skpref\metrics\_classification.py:194: RuntimeWarning: invalid value encountered in multiply
  np.nan_to_num(logged * binarized_outcome) +
../..\skpref\metrics\_classification.py:197: RuntimeWarning: divide by zero encountered in log
  logged2 = np.log(predicted2[_alternative])
../..\skpref\metrics\_classification.py:201: RuntimeWarning: invalid value encountered in multiply
  np.nan_to_num(logged2 * binarized_outcome) +
